# Testing envs

In [1]:
import os

print(os.getcwd())
if not os.getcwd().endswith("app"):
    os.chdir("../app")
    print(os.getcwd())

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

%load_ext autoreload
%autoreload 2
%matplotlib inline

/home/mique/Desktop/Code/rl-module/notebooks
/home/mique/Desktop/Code/rl-module/app


# 2048

In [2]:
from src.custom_envs import Env2048

env = Env2048()

In [3]:
state = env.reset()
state

array([[0, 0, 0, 0],
       [2, 0, 2, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int16)

In [4]:
state, reward, terminated, truncated, info = env.step(0)
state

AttributeError: 'Env2048' object has no attribute 'n_new_boxes_per_action'